In [1]:
import re
import os
import glob
import pandas as pd
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 2000
import time

time_string = time.strftime("%Y%m%d")

# Set the directory you want to start from
root_dir = '/home/pgajo/working/subtitling/subsplitter/data/aws/raw_srts' 

output_dir = '/home/pgajo/working/subtitling/subsplitter/data/aws/merged_srts'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def merge_srt_files(root_dir):
    # Get a list of all .srt files in the directory
    srt_files = glob.glob(os.path.join(root_dir, '*.srt'))

    # Sort the files in ascending order (optional)
    srt_files.sort()

    # Get the name of the directory
    dir_name = os.path.basename(output_dir)

    # Output file will be named as directory name with '_merged' suffix
    output_file = os.path.join(output_dir, f'{dir_name}_merged_{time_string}.srt')

    with open(output_file, 'w') as outfile:
        # Initialize subtitle index
        subtitle_index = 1
        for fname in srt_files:
            with open(fname) as infile:
                for line in infile:
                    # Check if line is a subtitle index
                    if line.strip().isdigit():
                        # Write the new subtitle index and increment it
                        outfile.write(str(subtitle_index) + '\n')
                        subtitle_index += 1
                    else:
                        # Write the line as is
                        outfile.write(line)

    # count number of \n\n\n in file
    with open(output_file, 'r') as infile:
        filedata = infile.read()
        # count = filedata.count('\n\n\n')
        # print(count)
    
    # remove triple \n
    with open(output_file, 'r') as infile:
        filedata = infile.read()
        filedata = filedata.replace('\n\n\n', '\n')

        # count number of \n\n\n in file
    with open(output_file, 'r') as infile:
        filedata = infile.read()
        # count = filedata.count('\n\n\n')
        # print(count)

    return output_file

output_file = merge_srt_files(root_dir)
print(output_file)

/home/pgajo/working/subtitling/subsplitter/data/aws/merged_srts/merged_srts_merged_20240120.srt


filter subtitles by length

In [2]:
nchars = 32
def read_srt_block(file):
    number = file.readline().rstrip()
    timestamp = file.readline().rstrip()
    lines = []
    while True:
        line = file.readline().rstrip()
        if line == '':
            break
        lines.append(line)
    return number, timestamp, lines

def write_srt_block(file, number, timestamp, lines):
    file.write(f"{number}\n")
    file.write(f"{timestamp}\n")
    for line in lines:
        file.write(f"{line}\n")
    file.write("\n")

def remove_formatting_tags(line):
    line_without_tags = re.sub('<[^>]*>', '', line)
    line_without_double_spaces = re.sub(' +', ' ', line_without_tags)
    return line_without_double_spaces

# output_file = '/home/pgajo/working/subtitling/subsplitter/data/merged_srt_singlefile/merged_srt_singlefile_merged.srt'

# define filtered output file name
output_file_merged = output_file[:-4] + f"_{nchars}.srt"

with open(output_file, "r", encoding='utf8') as unfiltered_file, open(output_file_merged, "w", encoding='utf8') as filtered_file:
    while True:
        number, timestamp, lines = read_srt_block(unfiltered_file)
        if number == '':
            break
        if all(len(remove_formatting_tags(line.strip())) <= nchars for line in lines):
            write_srt_block(filtered_file, number, timestamp, lines)
        else:
            print(lines)

# check number of lines that are longer than {nchars} characters in output_file and output_file_merged

with open(output_file, "r", encoding='utf8') as f:
    lines = f.readlines()
    print(f"Number of lines in {output_file}: {len(lines)}")
    print(f"Number of lines longer than {nchars} characters in {output_file}: {len([line for line in lines if len(remove_formatting_tags(line.strip())) > nchars])}")

with open(output_file_merged, "r", encoding='utf8') as g:
    lines = g.readlines()
    print(f"Number of lines in {output_file_merged}: {len(lines)}")
    print(f"Number of lines longer than {nchars} characters in {output_file_merged}: {len([line for line in lines if len(remove_formatting_tags(line.strip())) > nchars])}")

Number of lines in /home/pgajo/working/subtitling/subsplitter/data/aws/merged_srts/merged_srts_merged_20240120.srt: 62811
Number of lines longer than 37 characters in /home/pgajo/working/subtitling/subsplitter/data/aws/merged_srts/merged_srts_merged_20240120.srt: 0
Number of lines in /home/pgajo/working/subtitling/subsplitter/data/aws/merged_srts/merged_srts_merged_20240120_37.srt: 62811
Number of lines longer than 37 characters in /home/pgajo/working/subtitling/subsplitter/data/aws/merged_srts/merged_srts_merged_20240120_37.srt: 0


In [3]:
dir = '/home/pgajo/working/subtitling/subsplitter/data/aws/raw_srts'
for file in os.listdir(dir):
    with open(os.path.join(dir, file), "r", encoding='utf8') as f:
        while True:
            number, timestamp, lines = read_srt_block(f)
            if number == '':
                break
            if not all(len(remove_formatting_tags(line.strip())) <= nchars for line in lines):
                print(file, lines)

# prepare subtitles as a training dataset for T5

In [4]:
# extract subtitle lines and join them with \n

with open(output_file_merged, 'r', encoding='utf8') as f:
    content = f.read()

def extract_subtitles(text):
    blocks = text.strip().split('\n\n')
    subtitles = []

    for block in blocks:
        print('block', block)
        lines = block.split('\n')[1:]
        # print('lines', lines)
        subtitle_line = '|'.join([line for line in lines if not '-->' in line]) # | is used to split lines within the same subtitle
        subtitle_line = re.sub('♪', '#', subtitle_line)
        # print('subtitle_line', subtitle_line)
        if subtitle_line:
            if subtitles and not subtitles[-1][-1] in {'.', '!', '?', ']', '[', '♪', '<', '>'}:
                # print('check')
                subtitles[-1] += '•' + subtitle_line # • is used to separate subtitles
            else:
                subtitles.append(subtitle_line)
    return subtitles

subtitle_lines_w_lb_newlines=extract_subtitles(content)
print(len(subtitle_lines_w_lb_newlines))
subtitle_lines_w_lb_newlines[:10]

# create a new list of subtitles without |, i.e., a list of unsplit subtitles

def remove_lb(sub_list):
    sub_list_no_lb = []
    for sub in sub_list:
        sub_no_lb=re.sub('\|', ' ', sub)
        sub_no_lb=re.sub('\•', ' ', sub_no_lb)
        sub_list_no_lb.append(sub_no_lb)
    return sub_list_no_lb

subtitle_lines_no_lb=remove_lb(subtitle_lines_w_lb_newlines)

for i,line in enumerate(subtitle_lines_no_lb[0:10]):
    print(i,line)

# convert the two lists into a dataframe
df = pd.DataFrame({
    'NO_LB': subtitle_lines_no_lb,
    'LB': subtitle_lines_w_lb_newlines,
    })
display(df)

df['max_len'] = nchars
df['line_length'] = df['NO_LB'].apply(lambda x: len(x))
df['exceeds_max_len'] = df['line_length'] > df['max_len']
df['exceeds_max_len'] = df['exceeds_max_len'].astype(int)
display(df)
# save the dataframe
output_filename = output_file_merged[:-4] + '_prepped.csv'
df.to_csv(output_filename, index=False)
print('df saved to', output_filename)

block 1
00:00:02,197 --> 00:00:03,918
[parlano sovrapposti]
block 2
00:00:04,002 --> 00:00:05,234
…Di lasciarti libero.
block 3
00:00:05,317 --> 00:00:07,368
Qualche minuto prima
anche, se riusciamo.
block 4
00:00:07,451 --> 00:00:08,497
Va bene, va bene.
block 5
00:00:08,604 --> 00:00:11,866
Allora prima cosa,
siccome per noi è protocollo,
block 6
00:00:11,949 --> 00:00:13,567
ti condivido lo schermo
block 7
00:00:13,650 --> 00:00:19,233
con il protocollo
per la confidentiality.
block 8
00:00:19,733 --> 00:00:21,900
Questo è il nostro protocollo
che ti manderemo,
block 9
00:00:21,983 --> 00:00:25,501
comunque il discorso è che
ovviamente l'intervista è volontaria
block 10
00:00:25,585 --> 00:00:27,500
e ti ringraziamo
per la tua disponibilità.
block 11
00:00:29,726 --> 00:00:31,101
A meno che
tu non ci dia comunque
block 12
00:00:31,184 --> 00:00:33,801
il permesso di utilizzare
il tuo nome e informazioni,
block 13
00:00:33,884 --> 00:00:38,133
queste non compariranno
da nessuna parte

,NO_LB,LB
0,[parlano sovrapposti],[parlano sovrapposti]
1,…Di lasciarti libero.,…Di lasciarti libero.
2,"Qualche minuto prima anche, se riusciamo.","Qualche minuto prima|anche, se riusciamo."
3,"Va bene, va bene.","Va bene, va bene."
4,"Allora prima cosa, siccome per noi è protocollo, ti condivido lo schermo con il protocollo per la confidentiality.","Allora prima cosa,|siccome per noi è protocollo,•ti condivido lo schermo•con il protocollo|per la confidentiality."
...,...,...
3991,"Quindi il mio invito è proprio quello di: vai, divertiti, goditi al massimo questa esperienza.","Quindi il mio invito è proprio|quello di: vai, divertiti,•goditi al massimo|questa esperienza."
3992,"Se ti perdi qualche evento live ci saranno poi, ci sarà poi la possibilità di vederli on demand, e quindi direi buon re:Invent.","Se ti perdi qualche evento live|ci saranno poi,•ci sarà poi la possibilità|di vederli on demand,•e quindi direi buon re:Invent."
3993,"Mi raccomando, alla fine facci sapere che cosa ne pensi, lasciaci i feedback, dicci che cosa possiamo migliorare.","Mi raccomando, alla fine|facci sapere che cosa ne pensi,•lasciaci i feedback,|dicci che cosa possiamo migliorare."
3994,"E ancora, ciao e buon divertimento.","E ancora, ciao e buon divertimento."


,NO_LB,LB,max_len,line_length,exceeds_max_len
0,[parlano sovrapposti],[parlano sovrapposti],37,21,0
1,…Di lasciarti libero.,…Di lasciarti libero.,37,21,0
2,"Qualche minuto prima anche, se riusciamo.","Qualche minuto prima|anche, se riusciamo.",37,41,1
3,"Va bene, va bene.","Va bene, va bene.",37,17,0
4,"Allora prima cosa, siccome per noi è protocollo, ti condivido lo schermo con il protocollo per la confidentiality.","Allora prima cosa,|siccome per noi è protocollo,•ti condivido lo schermo•con il protocollo|per la confidentiality.",37,114,1
...,...,...,...,...,...
3991,"Quindi il mio invito è proprio quello di: vai, divertiti, goditi al massimo questa esperienza.","Quindi il mio invito è proprio|quello di: vai, divertiti,•goditi al massimo|questa esperienza.",37,94,1
3992,"Se ti perdi qualche evento live ci saranno poi, ci sarà poi la possibilità di vederli on demand, e quindi direi buon re:Invent.","Se ti perdi qualche evento live|ci saranno poi,•ci sarà poi la possibilità|di vederli on demand,•e quindi direi buon re:Invent.",37,127,1
3993,"Mi raccomando, alla fine facci sapere che cosa ne pensi, lasciaci i feedback, dicci che cosa possiamo migliorare.","Mi raccomando, alla fine|facci sapere che cosa ne pensi,•lasciaci i feedback,|dicci che cosa possiamo migliorare.",37,113,1
3994,"E ancora, ciao e buon divertimento.","E ancora, ciao e buon divertimento.",37,35,0


df saved to /home/pgajo/working/subtitling/subsplitter/data/aws/merged_srts/merged_srts_merged_20240120_37_prepped.csv
